# Procesado de modelos 3D

Este notebook guarda en un archivo csv las transformaciones que hay que realizar a un modelo para alinearlo

In [ ]:
# Imports
import trimesh
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import pyvista as pv
from PIL import Image
import vtk
import csv
import os

Variables globales

In [ ]:
# Nombre del archivo CSV
archivo_csv = "matrices_transformacion.csv"

if os.path.exists(archivo_csv):
    os.remove(archivo_csv)
    with open(archivo_csv, mode='w', newline='') as file:
        print("Borrado y creado .csv")

# Lista de índices de landmarks para nariz, ojos y boca
landmark_indices = [130, 27, 243, 23, 463, 257, 359, 253, 168, 61, 11, 291, 16]

# Crear un objeto FaceLandmarker
base_options = python.BaseOptions(model_asset_path='face_landmarker.task')
VisionRunningMode = mp.tasks.vision.RunningMode
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       running_mode=VisionRunningMode.IMAGE,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

### Funciones auxiliares

Función para sacar una imagen 2D dado un objeto en 3D

In [ ]:
def take_screenshot(actors, plane, colors = None, textures = None, position = None, window_size = [1024, 768]):
    
    if position is None:
        position = [0.0, 0.0, 0.0]

        if plane.lower() == "xy" or plane.lower() == "yx":
            position[2] = 1000.0
        elif plane.lower() == "yz" or plane.lower() == "zy":
            position[0] = 1000.0
        elif plane.lower() == "xz" or plane.lower() == "zx":
            position[1] = 1000.0 
        else:
            print("ERROR: Wrong plane", plane)
            exit(1)
        

    plotter = pv.Plotter(off_screen = True, window_size = window_size)
    plotter.set_background("white")

    if colors is None:
        colors = [None] * len(actors)

    if textures is None:
        textures = [None] * len(actors)

    for actor, color, tex  in zip(actors, colors, textures):
        _ = plotter.add_mesh(actor, color = color, texture = tex)

    plotter.set_position(position)
    plotter.set_focus([0.0, 0.0, 0.0])
    plotter.enable_parallel_projection()
    plotter.parallel_scale = 200

    return np.array(plotter.screenshot()), plotter

Función para pasar punto 2D a punto 3D

In [ ]:
def convert_2d_point_to_3d(pv_mesh, point, plotter):
    
    coordinate = vtk.vtkCoordinate()
    coordinate.SetCoordinateSystemToNormalizedDisplay()

    # the system coordinates is normalized, so 0.5 is the half of the image
    coordinate.SetValue(point[0], point[1])
    world_point = coordinate.GetComputedWorldValue(plotter.renderer)

    start = [world_point[0], -world_point[1], 1000]
    end = [world_point[0], -world_point[1], -1000]

    points, _ = pv_mesh.ray_trace(start, end)

    final_point = None

    if len(points) > 0:
        final_point = points[0]
    
    coordinate = None # Liberar memoria

    return final_point

Código para guardar los puntos en un .pp

In [ ]:
def guardar_pp(output_path, coordenadas_3d):
    with open(output_path, "w") as f:
        f.write("<!DOCTYPE PickedPoints>\n")
        f.write("<PickedPoints>\n")
        i = 0
        for idx in coordenadas_3d:
            x, y, z = idx[0], idx[1], idx[2]
            f.write(
                '<point x="{}" y="{}" z="{}" active="1" name="{}" />\n'.format(
                    x, y, z, i
                )
            )
            i += 1
        f.write("</PickedPoints>")

Código para obtener coordenadas de los puntos clave

In [ ]:
def obtener_coordenadas(obj_file_path, texture_file_path, output_path):
    # Obtener la imagen del modelo 3D
    pv_mesh = pv.read(obj_file_path)
    texture = pv.read_texture(texture_file_path)
    screenshot, plotter = take_screenshot([pv_mesh], plane='xy', textures=[texture])
    image2 = Image.fromarray(np.uint8(screenshot))
    image2.save('modelo_objetivo.png')

    # Detectamos los face landmarks de la imagen
    imagen = 'modelo_objetivo.png'
    image = mp.Image.create_from_file(imagen)
    detection_result = detector.detect(image)

    puntos = []
    for face_landmarks in detection_result.face_landmarks:
        for idx in landmark_indices:
            landmark = face_landmarks[idx]
            # Crear un vector con coordenadas x, y
            punto_2d = [landmark.x, landmark.y]
            # Agregar el vector a la lista
            puntos.append(punto_2d)
    
    # Calcular la coordenada z
    coordenadas_3d = []

    for point in puntos:
        # Convertir cada punto 2D a 3D utilizando la función proporcionada
        converted_point = convert_2d_point_to_3d(pv_mesh, point, plotter)
        # Agregar el punto convertido a la lista
        coordenadas_3d.append(converted_point)
        
    # print("Coordenadas 3D resultantes:", coordenadas_3d)
    guardar_pp(output_path, coordenadas_3d)

    # Liberar memoria
    pv_mesh = None
    image = None
    image2 = None
    plotter = None

    return coordenadas_3d


Función para guardar una matriz en el archivo CSV

In [ ]:
def guardar_matriz_en_csv(nombre, matriz, archivo_csv):
    if not isinstance(matriz, np.ndarray):
        raise TypeError("La matriz debe ser una lista de listas")
    
    try:
        # Apertura del archivo en modo append ('a')
        with open(archivo_csv, mode='a', newline='') as file:
            writer = csv.writer(file)
            valores_matriz = [valor for fila in matriz for valor in fila]
            writer.writerow([nombre] + valores_matriz)
    except IOError as e:
        print(f"Error al escribir en el archivo CSV: {e}")

### Obtener puntos clave modelo referencia

Cargamos el modelo de referencia 3D

In [ ]:
# Ruta del archivo .obj
obj_file_path = 'referencia H3DS-net/referencia.obj'
texture_file_path = 'referencia H3DS-net/referencia.png'
pp_file_path = 'referencia H3DS-net/referencia.pp'

coord_ref = obtener_coordenadas(obj_file_path, texture_file_path, pp_file_path)

### Obtener matriz de transformación para otro modelo

In [ ]:
archivos = []
ruta_carpeta = os.getcwd() + "/results/HeadSpace-final"

print(ruta_carpeta)
# ruta_carpeta = os.getcwd()
for root , _, archs in os.walk(ruta_carpeta):
    for archivo in archs:
        # Verificar si el archivo tiene la extensión .obj
        if archivo.endswith(".obj"):
            ruta_archivo = os.path.join(root, archivo)
            nombre, extension = os.path.splitext(ruta_archivo)
            ruta_relativa = os.path.relpath(nombre, os.getcwd())
            archivos.append(ruta_relativa)
            print(ruta_relativa)

In [ ]:
for archivo in archivos:
    # Ruta del archivo .obj
    obj_file_path = archivo + ".obj"
    pp_file_path = archivo + ".pp"
    texture_file_path = archivo  + ".bmp"

    coord_obj = obtener_coordenadas(obj_file_path, texture_file_path, pp_file_path)

    v0 = list(zip(*coord_ref))
    v1 = list(zip(*coord_obj))
    matriz_transformacion = trimesh.transformations.affine_matrix_from_points(v0, v1, shear=False,scale=False)

    for i in range(3):
        matriz_transformacion[i][3] *= -1

    # print(matriz_transformacion)

    guardar_matriz_en_csv(obj_file_path, matriz_transformacion, archivo_csv)
    print(obj_file_path)